In [183]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
import pickle

train_data = pd.read_csv('dataset/fakenewstrain.csv')
for i,x in enumerate(train_data.columns):
    if i!=0:
        print(i-1,x)
test_data = pd.read_csv('dataset/fakenewstest.csv')

# Split data into features and labels
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values



X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values
#print(X_train)
X_train = X_train[:,1:]
X_test = X_test[:,1:]


# replace NaN values with 0 in X
X_train = np.nan_to_num(X_train)

X_test = np.nan_to_num(X_test)


0 quotes
1 exclaim
2 allpunc
3 allcaps
4 stops
5 CC
6 CD
7 DT
8 EX
9 FW
10 IN
11 JJ
12 JJR
13 JJS
14 LS
15 MD
16 NN
17 NNS
18 NNP
19 NNPS
20 PDT
21 POS
22 PRP
23 PRP$
24 RB
25 RBR
26 RBS
27 RP
28 SYM
29 TO
30 UH
31 WP$
32 WRB
33 VB
34 VBD
35 VBG
36 VBN
37 VBP
38 VBZ
39 WDT
40 WP
41 $
42 ''
43 (
44 )
45 ,
46 --
47 .
48 :
49 ``
50 ttr
51 avg_wordlen
52 word_count
53 flesch_kincaid_grade_level
54 smog_index
55 coleman_liau_index
56 lix
57 bias_words
58 assertatives
59 factives
60 hedges
61 implicatives
62 report_verbs
63 positive_opinion_words
64 negative_opinion_words
65 vadneg
66 vadneu
67 vadpos
68 wneg
69 wpos
70 wneu
71 sneg
72 spos
73 sneu
74 HarmVirtue
75 HarmVice
76 FairnessVirtue
77 FairnessVice
78 IngroupVirtue
79 IngroupVice
80 AuthorityVirtue
81 AuthorityVice
82 PurityVirtue
83 PurityVice
84 MoralityGeneral
85 num_locations
86 num_dates
87 label


In [176]:
best_score = -1
best_a = -1
best_b = -1
for i in range(1,2):
    a = random.randrange(len(X_train[0]))
    b = random.randrange(len(X_train[0]))
    X_train2 = X_train[:,[24, 22, 4, 86]]
    X_test2 = X_test[:,[24, 22, 4, 86]]
    #knn = KNeighborsClassifier(n_neighbors=i)
    #knn.fit(X_train, [1 if x > 0.5 else 0 for x in y_train])
    #predictions = knn.predict(X_test)
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(5, 2), random_state=1)
    clf.fit(X_train2, y_train)
    predictions = clf.predict(X_test2)
    score = sum(y_test==predictions)/len(y_test)
    if score > best_score:
        best_score = score
        best_a = a
        best_b = b
print(best_a,best_b,best_score)

51 20 0.782


In [177]:
scores = []
best_score = -1
best_a = -1
best_b = -1
best_i = -1
for i in tqdm(range(1,500)):
    a = random.randrange(len(X_train[0]))
    b = random.randrange(len(X_train[0]))
    X_train2 = X_train[:,[a,b]]
    X_test2 = X_test[:,[a,b]]
    X_train2 = X_train[:,[24, 22, 4, 86]]
    X_test2 = X_test[:,[24, 22, 4, 86]]
    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train2, y_train)
    predictions = knn.predict(X_test2)
    score = sum(y_test==predictions)/len(y_test)
    scores.append((a,b,score))
    if score > best_score:
        best_score = score
        best_a = a
        best_b = b
        best_i = i
print(best_a,best_b,best_score)
#print(y_test)
#print(predictions)


100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [00:08<00:00, 59.11it/s]

12 18 0.8


In [178]:
X_train2 = X_train[:,[24, 22, 4, 86]]
X_test2 = X_test[:,[24, 22, 4, 86]]
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train2, y_train)
with open('knnfakenews.pkl','wb') as f:
    pickle.dump(knn,f)

In [179]:
scores.sort(key=lambda x: x[2],reverse=True)
gscores = list(filter(lambda x: x[2] >= 0.7,scores))

In [180]:
indices = [x[0] for x in gscores] + [x[1] for x in gscores]

In [181]:
counts = Counter(indices).most_common()
gcounts = list(filter(lambda x: x[1] > 2,counts))
print([x[0] for x in gcounts])

[79, 0, 42, 13, 57, 49, 58, 56, 62, 59, 38, 76, 46, 63, 74, 6, 22, 33, 44, 75, 83, 32, 71, 41, 50, 15, 52, 69, 78, 70, 67, 19, 29, 18, 39, 64, 20, 7, 28, 80, 12, 23, 35, 72, 30, 37, 51, 77, 5, 68, 43, 25, 40, 53, 81, 73, 9, 3, 34, 55, 8, 60, 47, 45, 16, 1, 27, 11, 61, 65, 54, 17, 82, 31, 14, 85, 24, 48, 4, 66, 86, 10, 2, 36, 21, 26, 84]
